In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data with selected columns
df = pd.read_csv('gym_churn_us.csv', usecols=[
    'gender', 'Near_Location', 'Partner', 'Promo_friends', 'Phone',
    'Contract_period', 'Group_visits', 'Age', 'Avg_additional_charges_total',
    'Month_to_end_contract', 'Lifetime', 'Avg_class_frequency_total',
    'Avg_class_frequency_current_month', 'Churn'
])

# Handle missing values if necessary (here filling with mean for numerical columns)
df.fillna(df.mean(), inplace=True)

# Convert categorical columns to dummy variables
df = pd.get_dummies(df, drop_first=True)

# Split data into features (X) and target (y)
X = df.drop('Churn', axis=1)
y = df['Churn']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

# Train a random forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"ROC AUC: {roc_auc}")
print(classification_report(y_test, y_pred))


Accuracy: 0.9141666666666667
ROC AUC: 0.871557557093502
              precision    recall  f1-score   support

           0       0.93      0.96      0.94       897
           1       0.86      0.79      0.82       303

    accuracy                           0.91      1200
   macro avg       0.90      0.87      0.88      1200
weighted avg       0.91      0.91      0.91      1200



In [ ]:
from sklearn.model_selection import GridSearchCV

# Define a grid of hyperparameters
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Set up grid search
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

print(f"Best parameters: {grid_search.best_params_}")


In [ ]:
# Feature importance in Random Forest
feature_importance = model.feature_importances_
features = X.columns

# Print the importance of each feature
for feature, importance in zip(features, feature_importance):
    print(f"{feature}: {importance}")

In [ ]:
import joblib

# Save the model to a file
joblib.dump(model, 'gym_churn_model.pkl')